# Problem might lie in loss! 
# Noooooo! Problem is not the loss! 

In [1]:
import torch
import torch.nn as nn

In [2]:
ls = nn.MSELoss(reduction='none')

In [3]:
mask = torch.tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 0.],
        [1., 0., 0.]])

In [4]:
x = torch.rand(5, 3, 2)
y = torch.rand(5, 3, 2)
# mask = torch.ones(128, 3)

In [5]:
loss = ls(x, y)

In [6]:
loss.shape

torch.Size([5, 3, 2])

In [7]:
mask.shape

torch.Size([5, 3])

In [8]:
b, t, f = y.shape

In [9]:
maskex = mask.unsqueeze(-1).expand((b, t, f)).bool()

In [10]:
maskex

tensor([[[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True],
         [False, False]],

        [[ True,  True],
         [False, False],
         [False, False]]])

In [11]:
loss.masked_select(maskex).mean()

tensor(0.1928)

In [12]:
y_hat_masked = y.masked_select(maskex)
y_masked = x.masked_select(maskex)

In [13]:
loss_fn = nn.MSELoss(reduction='mean')

In [14]:
loss = loss_fn(y_hat_masked, y_masked)

In [15]:
loss

tensor(0.1928)

In [16]:
# loss_fn = nn.MSELoss(reduction='none')

In [17]:
def masked_loss(loss_fn, y_hat, y, mask):
    """
    Computes the masked loss given a loss function, predicted values, ground truth values, and a mask tensor.
    """
    b, t, f = y_hat.shape
    mask = mask.unsqueeze(-1).expand((b, t, f)).bool()  # both y_hat and y are of same size
    # Apply mask to the predicted and ground truth values
    y_hat_masked = y_hat.masked_select(mask)
    y_masked = y.masked_select(mask)

    # Calculate the loss using the masked values
    loss = loss_fn(y_hat_masked, y_masked)
    # loss = torch.sum(loss_fn(y_hat_masked, y_masked)) / torch.sum(mask)

    return loss

In [18]:
def ed_loss(loss_fn, y_hat, y, mask):
    """
    Computes the masked loss given a loss function, predicted values, ground truth values, and a mask tensor.
    """
    b, t, f = y_hat.shape
    mask = mask.unsqueeze(-1).expand((b, t, f)).bool()  # both y_hat and y are of same size
    # Apply mask to the predicted and ground truth values
    # y_hat_masked = y_hat.masked_select(mask)
    # y_masked = y.masked_select(mask)

    # Calculate the loss using the masked values
    # loss = loss_fn(y_hat_masked, y_masked)
    # loss = torch.sum(loss_fn(y_hat_masked, y_masked)) / torch.sum(mask)
    loss = loss_fn(y_hat, y)

    return loss

In [19]:
masked_loss(loss_fn, y, x, mask)

tensor(0.1928)

In [20]:
ed_loss(loss_fn, y, x, mask)

tensor(0.1855)

In [87]:
torch.sum(mask)

tensor(384.)

In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib as plt

In [2]:
y, sr = librosa.load('s2201b_00000935.wav', sr = None)

In [4]:
normalizedy = librosa.util.normalize(y)

stft = librosa.core.stft(normalizedy, n_fft = 256, hop_length=16)
mel = librosa.feature.melspectrogram(S=stft, n_mels=32)
mellog = np.log(mel + 1e-9)
melnormalized = librosa.util.normalize(mellog)

In [7]:
melnormalized.shape

(32, 714)

In [ ]:
# Loop through each dimension and plot the histogram
for i in range(dim):
    plt.figure()
    plt.hist(x[:, :, i].flatten(), bins=20)
    plt.title(f"Histogram for dimension {i+1}")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.show()